In [1]:
import logging
import torch
import numpy as np
from transformers import AutoTokenizer, Trainer
from datasets import Dataset, ClassLabel

/home/jonhue/projects/jonhue/twitter-sentiment-classification/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from loading import load_train
from preprocessing import preprocess
from evaluation import evaluate
from bert import tokenize, get_BERT

[nltk_data] Downloading package stopwords to /home/jonhue/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jonhue/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
logging.basicConfig(level=logging.INFO)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
FULL=False
EVAL_FRAC = 0.2

MODEL = 'base-small'
TOKENIZER = 'bert-base-uncased'
PREPROCESSING = None

In [6]:
_, df_eval = load_train(full=FULL, eval_frac=EVAL_FRAC, x_col='text', y_col='label', neg_label=0, pos_label=1)
preprocess(df_eval, flags=PREPROCESSING, x_col='text')
dataset_eval = Dataset.from_pandas(df_eval)

new_features = dataset_eval.features.copy()
new_features['label'] = ClassLabel(names=['0', '1'])

dataset_eval = dataset_eval.cast(new_features)

Casting the dataset: 100%|██████████| 4/4 [00:00<00:00, 18.96ba/s]


In [7]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER)
eval_tokenized = tokenize(dataset_eval, tokenizer, path=f'bert/cache/eval_tokenized__{TOKENIZER}__{EVAL_FRAC}{"__full" if FULL else ""}')

In [8]:
model = get_BERT(MODEL, device)

In [9]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return evaluate(labels, predictions)

trainer = Trainer(model, eval_dataset=eval_tokenized, tokenizer=tokenizer, compute_metrics=compute_metrics)

In [10]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: index, text, token_type_ids, __index_level_0__. If index, text, token_type_ids, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40000
  Batch size = 8


INFO:root:---
* accuracy: 0.9044
* precision: 0.8971451111878113
* recall: 0.9128079064867306
* f1: 0.9049087382503606
* bce: 3.3019487223594464
* auc: 0.904428072444043
---


{'eval_loss': 0.2327585518360138,
 'eval_accuracy': 0.9044,
 'eval_precision': 0.8971451111878113,
 'eval_recall': 0.9128079064867306,
 'eval_f1': 0.9049087382503606,
 'eval_bce': 3.3019487223594464,
 'eval_auc': 0.904428072444043,
 'eval_runtime': 185.464,
 'eval_samples_per_second': 215.675,
 'eval_steps_per_second': 26.959}